# Household

## 1. Data/settings

If the local variable $t$ has been defined already, the notebook is running from another notebook. If this is not the case, a number of "local" methods have to be carried out before solving the model (e.g. setting up the relevant IO data set):

In [1]:
try:
    t = t
except NameError:
    clean_up = True
    %run StdPackages.ipynb
    name, t = 'full', 2019
    ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
    with open(f"{d['processedData']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
        glob=pickle.load(file)
    db_IO = GpyDB(pickle_path = os.path.join(d['processedData'], f'{name}_{t}')) # load IO database named IO_name
    addTimeToDb(db_IO, glob.db['t'].vals, exceptions = ['sigma'])
module = '_'.join([name,str(t),'HH'])
db_IO0 = db_IO.copy() # create copy

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.


## 2. Initial values

Set up nesting tree using the nesting structure:

In [2]:
tree = nestingTree.tree(name = f'{module}_ces', tree = db_IO.get('nestHH').to_list()) # individual tree
Tree = nestingTree.aggTree(name = module, trees = {tree.name: tree})(namespace = {str(n)+'_input': n for n in db_IO.get('n')})

## 3. Run module

*1. Initialize production module, without any durables at first:*

In [3]:
H = mHouseholds.staticConsumer(tree = Tree, glob = glob, kwargs = {'L2C': db_IO.get('L2C')})
aggregateDB.subset_db(db_IO0, H.get('s')) # goes through all symbols in db_IO and only keep the elements that are in the set 's' from Tree.db
aggregateDB.subset_db(db_IO0, H.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
# robust.robust_merge_dbs(H.s.db, db_IO, priority = 'second')

*2. Add value shares:*

Value shares for nesting part:

In [4]:
v = valueShares.SimpleRamsey(Tree, db_IO0)
v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
v.write(); # write GAMS code used for the model
m = v.run(exportTo = d['work'],ws=ws) # solve the "model".

Use value shares to initialize variables:
* Outputs and inputs are provided by IO data.
* For intermediate goods, assume a price of 1 (default option in the class, so we don't have to do anything) and set value share = quantity.
* Set share parameters to the ones identified by value share system.

In [5]:
gpyDB.GpyDBs_AOM_Second(H.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), H.get('int')).rename('qD'))) # set intermediate goods levels
gpyDB.GpyDBs_AOM_Second(H.s.db, gpy(m.out_db.get('mu').xs(H.get('t0')[0]).rename('mu'))) # set share parameters

Add the frisch parameter as a target in the IO database if we want to have an intensive margin labor elasticity in there:

In [6]:
# db_IO['frisch'] = pd.Series(0.25, index = H.get('labor'), name = 'frisch')

Use sneaky calibration method to solve and calibrate:

In [7]:
H.compile(initDB=True)
H.write();
mCalib, cp = H.sneakyCalib(db_IO.copy(), ws = ws, loop_kwargs = {'n': 25}, **{'cns': 'CONOPT4'})

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\sxj477\Documents\GitHub\CGE_Generator\work_folder\_gams_py_gjo3.lst for more details

### 4. Export 

*Use data from the calibration step in the main database. The ```sneakyCalib``` method defines some new, auxiliary symbols - these are not used.*

In [ ]:
[gpyDB.GpyDBs_AOM_Second(H.s.db, mCalib.out_db[k]) for k in H.s.db.symbols];
H.s.setstate('B')

Export:

In [ ]:
with open(os.path.join(d['modules'], H.name), "wb") as file:
    pickle.dump(H,file)